In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# GRU
<img src="https://velog.velcdn.com/images%2Fguide333%2Fpost%2F5c9b1e3c-86c4-481a-99b3-49457f6aa362%2FScreenshot%20from%202021-04-24%2016-45-38.png">


+ GRU_Cell
    + input_size : input data의 size (embedding을 할 경우 embedding의 size이다)
    + hidden_size : hidden state의 차원 수
    + bias : 각 식의 존재하는 편향 값


+ if x 가 m*n의 차원이면 h차원가 맞게 weight를 곱하여 m*h의 차원으로 변경한다.
+ Weight가 x2x 2개 x2h 2개가 필요하므로 각각 한번에 2개를 만들어서 잘라서 사용
+ 데이터 example을 시계열 데이터로 사용 [batch, sequece, variable] 3차원 데이터
+ GRU 계산에서는 행렬 계산 뿐만 아니라 행렬 요소 곱도 포함 된다.
+ candiate r_t ,h_t-1의 행렬요소 곱을 위해 size를 hidden size로 맞춰주어야 한다
+ hidden_state 차원 [batch, Layer, hidden]

In [2]:
class GRU_Cell(nn.Module):
    def __init__(self,input_size,hidden_size,bias=True):
        super().__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.bias=bias
        self.x2h=nn.Linear(input_size,hidden_size*2,bias=bias)
        self.h2h=nn.Linear(hidden_size,hidden_size*2,bias=bias)
        self.x2c=nn.Linear(input_size,hidden_size,bias=bias)
        self.c2c=nn.Linear(hidden_size,hidden_size,bias=bias)
    def forward(self,x,hidden_state):
        # x : [batch, variable]
        gates=self.x2h(x)+self.h2h(hidden_state)
        update_gate,reset_gate=gates.chunk(2,dim=1)
        candidate=torch.mul(reset_gate,hidden_state)
        
        
        candidate_Hidden_state=F.tanh(self.x2c(x)+self.x2h(torch.mul(reset_gate,hidden_state)))
        
        hidden_state=torch.mul(update_gate,hidden_state)+torch.mul((1-update_gate),candidate_Hidden_state)
        
        return hidden_state

        
        

# Multi_Layer

+ Mult_layer 적용 시 2번 째 cell부터는 입력의 차원이 [batch, hidden_size]이므로 선언을 새로 해주어야 한다.

+ Layer 수만큼 반복하면서 각 계층 별 hidden_state를 저장한 후에 다음 시점으로 보내주어야 한다.

In [3]:
class GRU_Model(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,Layer_num=1) :
        super().__init__()
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.output_dim=output_dim
        self.Layer_num=Layer_num
        self.GRU_Cell=GRU_Cell(input_dim,hidden_dim)
        self.fc1=nn.Linear(hidden_dim,output_dim)
        self.GRU_MultiLayer=GRU_Cell(hidden_dim,hidden_dim)
    
    
    def forward(self,x):
        # x [batch,seq,variable]
        # hidden_state [batch,Layer_num,hidden_dim]
        cyc=x.size(1)
        hidden_state=torch.zeros(x.size(0),self.Layer_num,self.hidden_dim)
        if self.Layer_num==1:
            # hidden_state [batch,1,hidden_dim]
            hidden_state.squeeze()
            # hidden_state [batch,hidden_dim]
            for seq in range(cyc):
                hidden_state=self.GRU_Cell(x,hidden_state)
            out_put=hidden_state
        else:
            # hidden_state [batch,Layer_num,hidden_dim]
            for seq in range(cyc):
                hidden_state[:,0,:]=self.GRU_Cell(x,hidden_state[:,0,:])
                for i in range(self.Layer_num-1):
                    hidden_state[:,i+1,:]=self.GRU_MultiLayer(hidden_state[:,i,:],hidden_state[:,i+1,:])
            output=hidden_state[:,-1,:]
         
        out_put=self.fc1(out_put)
        return out_put   
    